In [1]:
#Importing the packages
import pandas as pd
import numpy as np
from sklearn.feature_extraction. text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import nltk,spacy
import re,string
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("C:\\Users\\Admin\\Downloads\\twitter_data.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head(5)

C:\Users\Admin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(24783, 7)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
#check the distribution of labels 
df['class'].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [4]:
df["labels"] = df['class'].map({0: "Hate Speech", 1: "Offensive Speech", 2: "No Hate and Offensive Speech"})
df = df[["tweet", "labels"]]

In [5]:
 def preprocess(text):
        test_list = text.split()
        text = ""
        for word in test_list:
            if word.endswith("ing"):
                word=word[:-3]
            text+=word
            text+=" "
        doc = nlp(text)
        filtered_tokens = []
        
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            filtered_tokens.append(token.lemma_)
            
        return " ".join(filtered_tokens)

In [6]:
df["tweet_new"] = df.tweet.apply(preprocess)

In [7]:
df.head()

,tweet,labels,tweet_new
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech,RT @mayasolovely woman complain clean house am...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech,RT @mleew17 boy dat cold tyga dwn bad cuffin d...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech,RT @UrKindOfBrand Dawg RT @80sbaby4life fuck b...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech,RT @C_G_Anderson @viva_base look like tranny
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech,RT @shenikaroberts shit hear true faker bitch ...


In [8]:
def clean(text):
    text = str (text). lower()
    text = re. sub('[.?]', '', text)
    text = re. sub('https?://\S+|www.\S+', '', text)
    text = re. sub('<.?>+', '', text)
    text = re. sub('[%s]' % re. escape(string. punctuation), '', text)
    text = re. sub('\n', '', text)
    text = re. sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ". join(text)
    text = [stemmer. stem(word) for word in text. split(' ')]
    text=" ". join(text)
    return text

In [9]:
df["tweet_new"] = df.tweet_new.apply(clean)

In [10]:
#df['vector'] = df['tweet_new'].apply(lambda text: nlp(text).vector)  

In [11]:
x = np. array(df["tweet_new"])
y = np. array(df["labels"])
cv = CountVectorizer()
X = cv.fit_transform(x)
df.head()

,tweet,labels,tweet_new
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech,rt mayasolov woman complain clean hous amp man...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech,rt mlee boy dat cold tyga dwn bad cuffin dat h...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech,rt urkindofbrand dawg rt babif fuck bitch star...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech,rt cganderson vivabas look like tranni
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech,rt shenikarobert shit hear true faker bitch te...


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = DecisionTreeClassifier()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
y_pred#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "Get the hell out of here"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

0.8822594449199168
hell
['Offensive Speech']


In [13]:
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = MultinomialNB()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
y_pred#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "Oh shit! I couldn't do that"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

0.8684435750091698
oh shit
['Offensive Speech']


In [14]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = RandomForestClassifier()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
y_pred#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "hi folks"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

0.8924073847658638
hi folk
['No Hate and Offensive Speech']


In [15]:
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = SVC()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
y_pred#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "What the hell!"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

0.9024330602763174
hell
['Offensive Speech']


In [16]:
from sklearn.ensemble import VotingClassifier

# Initialize individual models
dt_model = DecisionTreeClassifier()
rf_model = RandomForestClassifier()
svm_model = SVC(kernel='linear', probability=True)  # Enable probability estimation for SVM

# Define the ensemble of models
ensemble_model = VotingClassifier(estimators=[
    ('decision_tree', dt_model),
    ('random_forest', rf_model),
    ('svm', svm_model)
], voting='soft')  # Use soft voting for probability averaging

# Training the ensemble model
ensemble_model.fit(X_train, y_train)

# Testing the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test)

# Accuracy Score of ensemble model
print("Ensemble Accuracy Score:", accuracy_score(y_test, y_pred_ensemble))

Ensemble Accuracy Score: 0.8970534295146105


In [21]:
# Get input from the user
user_input = input("Enter the text you want to classify: ")

# Preprocess the user input
user_input_processed = preprocess(user_input)

# Transform the preprocessed input using the TF-IDF vectorizer
user_input_vectorized = cv.transform([user_input_processed]).toarray()

# Make prediction using the ensemble model
prediction = ensemble_model.predict(user_input_vectorized)

# Print the predicted label
print("Predicted label:", prediction)

Enter the text you want to classify: hello
Predicted label: ['No Hate and Offensive Speech']
